In [ ]:
#!/usr/bin/env


"""


make line datacubes and integrated maps
and compare applying continuum selfcal
NOTE: this is intended to be an interactive, iterative process
      so this is more a log that should be run by cutting and
      pasting into casa rather than as an executable script
      search "CHANGEME" for variables to be changed
10/9/15 MCA
"""


#  Setup 

III_88 SpT? II
16:09:20.30 -39:04:01.6


In [ ]:
field = 35                                     # CHANGEME 
fitspw    = '2,3,4'                                         # line-free channels for fitting continuum
linespw   = '0,1,3'                                         # line spectral windows (C18O, 13CO, CN)
robust     = 0.5                                            # CHANGEME
imsize     = [640,640]
cell       = '0.03arcsec'
imagermode = 'csclean'
outframe   = 'lsrk'
veltype    = 'radio'
width      = '1.0km/s'
start      = '-3km/s'
nchan      = 16
xc         = 320                               # CHANGEME
yc         = 320                               # CHANGEME
in_a       = 80
out_a      = 120
aper       = 0.5
boxwidth = 300.
box = rg.box([xc-boxwidth,yc-boxwidth],[xc+boxwidth,yc+boxwidth])


#  Create continuum subtracted line datasets 

In [ ]:
uvcontsub(vis       = 'f'+str(field)+'.vis',                # full vis file for this field
          spw       = linespw,                              # line spw (for cont subtraction)
          fitspw    = fitspw,                               # cont spw
          combine   = 'spw',
          solint    = 'int',
          fitorder  = 1,
          want_cont = False)                                # should not be changed.


#  13CO line 

first try on 13CO line as thats going to be the brightest
os.system('rm -rf f'+str(field)+'_13co32*')


In [ ]:
clean(vis           = 'f'+str(field)+'.vis.contsub',
      imagename     = 'f'+str(field)+'_13co32',
      mode          = 'velocity',
      start         = start,
      width         = width,
      nchan         = nchan,
      outframe      = outframe,
      veltype       = veltype,
      restfreq      = '330.58797GHz',
      niter         = 100,
      threshold     = 0,
      interactive   = False,
      imsize        = imsize,
      cell          = cell,
      weighting     ='briggs',
      robust        = robust,
      imagermode    = imagermode)


can't see gas or continuum emission
cleaned in automated mode without mask
export cube to fits file    


In [ ]:
fbase = 'f'+str(field)+'_13co32'
os.system('rm -rf '+fbase+'.cube.fits')
exportfits(imagename=fbase+'.image',fitsimage=fbase+'.cube.fits')


redo moment0 maps (now excluding noisy first channel!!)
os.system('rm -rf f'+str(field)+'_13co32.mom0*')


In [ ]:
immoments(imagename  = 'f'+str(field)+'_13co32.image',          # CHANGEME (based on above analysis!!)
          outfile    = 'f'+str(field)+'_13co32.mom0',
          moments    = [0],
          includepix = [-10.0,100.0],
          chans      = ('range=[2km/s,6km/s]'))                      # CHANGEME (based on above analysis!!)


export to fits file    


In [ ]:
fbase = 'f'+str(field)+'_13co32.mom0'
os.system('rm -rf '+fbase+'.fits')
exportfits(imagename=fbase,fitsimage=fbase+'.fits')


measure flux


In [ ]:
im_max = imstat(imagename = 'f'+str(field)+'_13co32.mom0')['max'][0]
im_rms = imstat(imagename = 'f'+str(field)+'_13co32.mom0',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
im_flux = imstat(imagename = 'f'+str(field)+'_13co32.mom0',
                 region='circle[['+str(xc)+'pix,'+str(yc)+'pix],'+str(aper)+'arcsec]')['flux'][0]
print 'Flux = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_flux, 1000*im_rms, im_flux/im_rms)


Flux = -119.74 mJy, rms = 46.14 mJy, S/N = -2.6
view gas (continuum not detected)


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_13co32.mom0'}])


re-center image on source and use measure.py to get COG flux
os.system('rm -rf f'+str(field)+'_13co32.mom0_cropped*')


In [ ]:
ia.fromimage(outfile = 'f'+str(field)+'_13co32.mom0_cropped.image',
             infile  = 'f'+str(field)+'_13co32.mom0.fits',
             region  = box )
ia.close() 
exportfits(imagename = 'f'+str(field)+'_13co32.mom0_cropped.image',
           fitsimage = 'f'+str(field)+'_13co32.mom0_cropped.fits')


In [ ]:
Measuring COG for G/f35_13co32.mom0_cropped.fits
Assuming object center (300.0,300.0)
Background: -1.43 mJy/beam km/s
RMS in annulus 4.0-6.0 arcsec = 43.64 mJy/beam km/s
   i   radius    flux      err       snr
       (asec)    (mJy)     (mJy)
   0     0.10    -6.46    11.94     -0.5
   1     0.20   -16.22    24.37     -0.7
   2     0.30   -30.41    43.46     -0.7 <-- ND
   3     0.40   -66.94    52.58     -1.3
   4     0.50  -119.74    78.65     -1.5
   5     0.60  -163.68    76.01     -2.2
   6     0.70  -197.45    87.54     -2.3
   7     0.80  -198.36    99.88     -2.0
F = -6.46 mJy
E = 11.94 mJy
S = -0.54
D = 0.20 arcsec


#  C18O line  

don't bother with selfcal as it doesn't help...
os.system('rm -rf f'+str(field)+'_c18o32*')


In [ ]:
clean(vis           = 'f'+str(field)+'.vis.contsub',
      imagename     = 'f'+str(field)+'_c18o32',
      mode          = 'velocity',
      start         = start,
      width         = width,
      nchan         = nchan,
      outframe      = outframe,
      veltype       = veltype,
      restfreq      = '329.33055GHz',
      niter         = 100,
      threshold     = 0,
      interactive   = False,
      imsize        = imsize,
      cell          = cell,
      weighting     ='briggs',
      robust        = robust,
      imagermode    = imagermode)


can't see gas or continuum emission
cleaned in automated mode without mask
export cube to fits file    


In [ ]:
fbase = 'f'+str(field)+'_c18o32'
os.system('rm -rf '+fbase+'.cube.fits')
exportfits(imagename=fbase+'.image',fitsimage=fbase+'.cube.fits')


redo moment0 maps (now excluding noisy first channel!!)
os.system('rm -rf f'+str(field)+'_c18o32.mom0*')


In [ ]:
immoments(imagename  = 'f'+str(field)+'_c18o32.image',            # CHANGEME (based on above analysis!!)
          outfile    = 'f'+str(field)+'_c18o32.mom0',
          moments    = [0],
          includepix = [-10.0,100.0],
          chans      = ('range=[2km/s,6km/s]'))


export to fits file    


In [ ]:
fbase = 'f'+str(field)+'_c18o32.mom0'
os.system('rm -rf '+fbase+'.fits')
exportfits(imagename=fbase,fitsimage=fbase+'.fits')


measure flux


In [ ]:
im_rms = imstat(imagename = 'f'+str(field)+'_c18o32.mom0',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
im_flux = imstat(imagename = 'f'+str(field)+'_c18o32.mom0',
                 region='circle[['+str(xc)+'pix,'+str(yc)+'pix],'+str(aper)+'arcsec]')['flux'][0]
print 'Flux = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_flux, 1000*im_rms, im_flux/im_rms)


Flux = -25.56 mJy, rms = 55.04 mJy, S/N = -0.5
view gas (continuum not detected)


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_c18o32.mom0'}])


re-center image on source and use measure.py to get COG flux
os.system('rm -rf f'+str(field)+'_c18o32.mom0_cropped*')


In [ ]:
ia.fromimage(outfile = 'f'+str(field)+'_c18o32.mom0_cropped.image',
             infile  = 'f'+str(field)+'_c18o32.mom0.fits',
             region  = box )
ia.close() 
exportfits(imagename = 'f'+str(field)+'_c18o32.mom0_cropped.image',
           fitsimage = 'f'+str(field)+'_c18o32.mom0_cropped.fits')


In [ ]:
Measuring COG for G/f35_c18o32.mom0_cropped.fits
Assuming object center (300.0,300.0)
Background: -0.28 mJy/beam km/s
RMS in annulus 4.0-6.0 arcsec = 52.04 mJy/beam km/s
   i   radius    flux      err       snr
       (asec)    (mJy)     (mJy)
   0     0.10   -16.64    16.27     -1.0
   1     0.20   -38.98    32.20     -1.2
   2     0.30   -35.74    54.59     -0.7 <-- ND
   3     0.40   -23.36    67.80     -0.3
   4     0.50   -25.56    80.54     -0.3
   5     0.60   -41.77    85.32     -0.5
   6     0.70   -54.99    98.95     -0.6
   7     0.80   -43.65   109.34     -0.4
F = -16.64 mJy
E = 16.27 mJy
S = -1.02
D = 0.20 arcsec


#  Image CN  

don't bother with selfcal as it doesn't help...
os.system('rm -rf f'+str(field)+'_cn32_b4sc*')


In [ ]:
clean(vis           = 'f'+str(field)+'.vis.contsub',
      imagename     = 'f'+str(field)+'_cn32_b4sc',
      mode          = 'velocity',
      start         = start,
      width         = width,
      nchan         = nchan,
      outframe      = outframe,
      veltype       = veltype,
      restfreq      = '340.24777GHz',
      niter         = 100,
      threshold     = 0,
      interactive   = False,
      imsize        = imsize,
      cell          = cell,
      weighting     ='briggs',
      robust        = robust,
      imagermode    = imagermode)


can't see gas or continuum emission
cleaned in automated mode without mask
redo moment0 maps (now excluding noisy first channel!!)
os.system('rm -rf f'+str(field)+'_cn32.mom0*')


In [ ]:
immoments(imagename  = 'f'+str(field)+'_cn32_b4sc.image',            # CHANGEME (based on above analysis!!)
          outfile    = 'f'+str(field)+'_cn32.mom0',
          moments    = [0],
          includepix = [-10.0,100.0],
          chans      = ('range=[1km/s,7km/s]'))                      # CHANGEME (based on above analysis!!)


export to fits file    


In [ ]:
fbase = 'f'+str(field)+'_cn32.mom0'
os.system('rm -rf '+fbase+'.fits')
exportfits(imagename=fbase,fitsimage=fbase+'.fits')


measure flux


In [ ]:
im_rms = imstat(imagename = 'f'+str(field)+'_cn32.mom0',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
im_flux = imstat(imagename = 'f'+str(field)+'_cn32.mom0',
                 region='circle[['+str(xc)+'pix,'+str(yc)+'pix],'+str(aper)+'arcsec]')['flux'][0]
print 'Flux = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_flux, 1000*im_rms, im_flux/im_rms)


Flux = 57.36 mJy, rms = 38.32 mJy, S/N = 1.5
view gas (continuum not detected)


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_cn32.mom0'}])


re-center image on source and use measure.py to get COG flux
os.system('rm -rf f'+str(field)+'_cn32.mom0_cropped*')


In [ ]:
ia.fromimage(outfile = 'f'+str(field)+'_cn32.mom0_cropped.image',
             infile  = 'f'+str(field)+'_cn32.mom0.fits',
             region  = box )
ia.close() 
exportfits(imagename = 'f'+str(field)+'_cn32.mom0_cropped.image',
           fitsimage = 'f'+str(field)+'_cn32.mom0_cropped.fits')


CAN'T SEE GAS 
